In [1]:
from Pytri.PetriNet import PNP
from Pytri.ConditionalProb import paper_cc_model_all, prob_label, conditional_prob
from itertools import chain, combinations

In [2]:
# P(s)
def pr_l(label):
    if label not in _pr_l:
        _pr_l[label] = prob_label(reachability_graph, label)
    return _pr_l[label]

# P(s|b)
def pr_lb(label, background):
    if label+background not in _pr_lb:
        _pr_lb[label+background] = conditional_prob(reachability_graph, label, background)
    return _pr_lb[label+background]

# Max_b(P(s|b))
def max_B(label):
    if label not in _max_B:
         _max_B[label] = max(pr_lb(label, background) for background in all_labels-set([label]))
    return _max_B[label]

def equal(a, b):
    return abs(a-b) < 1e-10

In [3]:
# For showing the difference between odds change and absolute odds
# I.E P(s|b) vs P(s|b) - P(s)

def change_test(printing = True):
    s1_labels = []

    # Print all labels with P(s) = Max_b(P(s|b))
    if printing: print("P(s) = Max_B(P(s|b)):")
    for label in all_labels:
        if equal(pr_l(label), max_B(label)):
            if printing:print(label)
            s1_labels.append(label)


    s2_labels = []        
    
    # Print all labels with P(s) < Max_b(P(s|b))
    if printing: print("\nP(s) < Max_B(P(s|b)):") 
    for label in all_labels:
        if pr_l(label) < max_B(label):
            if printing: print(label)
            s2_labels.append(label)

    
    found = False
    
    # Print all labels with P(s2) < Max_B(P(s2|b) < p(s1) = Max_B(P(s1|b))
    if printing:print("\nP(s2) < Max_B(P(s2|b) < p(s1) = Max_B(P(s1|b)):")
    for label1 in s1_labels:
        for label2 in s2_labels:
            if max_B(label2) < pr_l(label1):
                found = True
                if printing: print(f"S1: {label1}, S2: {label2}")
    
    return found

In [4]:
# For showing the difference between using the worst case and many bad cases
# I.E max_b(P(s|b)) vs sum_b(P(s|b))

def worst_test(printing = True):
    probs = []
    
    lall_labels = list(all_labels)
    
    for label in lall_labels:
        probs.append(pr_l(label))
    sorted_probs = sorted(probs)
    
    
    found = False
    
    if printing: print("P(b1) < min_(B-b1)(P(b)):")
    if sorted_probs[0] < sorted_probs[1]:
        b1 = lall_labels[probs.index(sorted_probs[0])]
        if printing: print(b1)
        
        
        if printing: print("P(s|b1) > max_(B-b1)(P(s|b)):")      
        for s in all_labels-set([b1]):
            succes = True
            for b in all_labels-set([b1, s]):
                if pr_lb(s, b1) <= pr_lb(s, b):
                    succes = False
                    break

            if succes:
                if printing: print(f"B1: {b1}, S: {s}")
                found = True
    
    return found

In [5]:
# For showing why the probability for a background matters
# I.E P(s|b) vs P(s|b) * P(b) 

def background_test(printing = True):
    part1_labels = []

    # Print all label combinatons with P(B1) < P(B2)
    if printing: print("P(b1) < P(b2):")
    for label1 in all_labels:
        for label2 in all_labels:
            if pr_l(label1) < pr_l(label2):
                if printing: print(f"B1: {label1}, B2: {label2}")
                part1_labels.append([label1, label2])
    
    
    found = False
    
    if printing: print("\nP(b1) < P(b2) & P(S|b1) = P(S|b2) != 0:")
    # Print all labels with P(B1) < P(B2) and P(S|B1) = P(S|B2) != 0:
    for labels in part1_labels:
        for label in all_labels-set(labels):
            first = pr_lb(label, labels[0])
            second = pr_lb(label, labels[1])
            if (not equal(first, 0)) and (not equal(first, 1)):
                if equal(first, second):
                    if printing: print(f"B1: {labels[0]}, B2: {labels[1]}, S:{label}")
                    found = True
    
    return found

In [6]:
# For showing why many small increases are less bad than one big one
# I.E P(s|b) - P(s) vs (P(s|b) - P(s))^2
number_combined = 2 # how many smaller increases must be combined to create one larger one

def increase_test_background(printing = True):
    found = False
    
    lall_labels = list(all_labels)
    
    if printing: print("\nP(s1|b) + ... + P(si|b) = (sm|b):")
    for background in lall_labels:
        new_lall_labels = lall_labels.copy()
        new_lall_labels.remove(background)
        background_probs = [pr_lb(label, background) for label in new_lall_labels]
        
        for target_index in range(len(background_probs)):
            target = background_probs[target_index]
            
            all_indices = list(range(len(background_probs)))
            all_indices.remove(target_index)
            
            for comb in combinations(all_indices, number_combined):
                if equal(sum(background_probs[index] for index in comb), target): # If all combined equal target
                    if all((not equal(background_probs[index], 0)) for index in comb):  # If none of the odds are 0
                        if printing: print(f"b: {background}, sm: {new_lall_labels[target_index]} {''.join(f',s: {new_lall_labels[index]}' for index in comb)}")
                        found = True
    
    return found

In [7]:
# For showing why many small increases are less bad than one big one
# I.E P(s|b) - P(s) vs (P(s|b) - P(s))^2
number_combined = 2 # how many smaller increases must be combined to create one larger one

def increase_test_sensitive(printing = True):
    found = False
    
    lall_labels = list(all_labels)
    
    if printing: print("\nP(s|b1) + ... + P(s|bi) = P(s|bm):")
    for sensitive in lall_labels:
        new_lall_labels = lall_labels.copy()
        new_lall_labels.remove(sensitive)
        background_probs = [pr_lb(sensitive, background) for background in new_lall_labels]
        
        for target_index in range(len(background_probs)):
            target = background_probs[target_index]
            
            all_indices = list(range(len(background_probs)))
            all_indices.remove(target_index)
            
            for comb in combinations(all_indices, number_combined):
                if equal(sum(background_probs[index] for index in comb), target): # If all combined equal target
                    if all((not equal(background_probs[index], 0)) for index in comb):  # If none of the odds are 0
                        if printing: print(f"s: {sensitive}, bm: {new_lall_labels[target_index]} {''.join(f',b: {new_lall_labels[index]}' for index in comb)}")
                        found = True
    
    return found

In [8]:
# For showing why an increase from small is more important
# I.E P(s|b) vs P(s|b)/P(s)

def small_test(printing = True):
    part1_label_duos = []

    # Print all label combinatons with P(s1) > P(s2)
    if printing: print("P(s1) > P(s2):")
    for label1 in all_labels:
        for label2 in all_labels:
            if pr_l(label1) > pr_l(label2):
                if printing: print(f"S1: {label1}, S2: {label2}")
                part1_label_duos.append([label1, label2])
    
    
    found = False
    
    # Print all label combinatons with P(s1|b) - P(s1) = P(s2|b) - P(s2) != 0 
    if printing: print("\nP(s1) > P(s2) and P(s1|b) - P(s1) = P(s2|b) - P(s2) != 0 :")
    for duo in part1_label_duos:
        for background in all_labels-set([duo[0], duo[1]]):
            first = pr_lb(duo[0], background) - pr_l(duo[0])
            second = (pr_lb(duo[1], background) - pr_l(duo[1]))
            
            if not equal(first, 0):
                if equal(first, second):
                    if printing: print(f"S1: {duo[0]}, S2: {duo[1]}, B: {background}")
                    found = True
    
    return found

In [9]:
models = ["Direct-Split", "Direct-Split(Tau)", "Direct-Split(None-Direct)", "Direct-Split(None-Split(Tau))", "Split-Split", 
          "Web", "Fingers", "Fingers2", "Fingers3", "Fingers4", "Fingers5", "Fingers6"]
#models = ["Web"]
for model in models:
    pnp = PNP.read_text(f"Synthetic/{model}.pnp")
    reachability_graph = pnp.create_reachability_graph()
    all_labels = set(arc.transition.label for arc in chain.from_iterable(state.outgoing_arcs for state in reachability_graph.states))-{'tau', 'None'}
    
    _max_B = {}
    _pr_l = {}
    _pr_lb = {}
    
    c_res = change_test(False)                # change in sensitive vs highest
    w_res = worst_test(False)                 # worst case vs all cases
    b_res = background_test(False)            # probability of the background matering
    ib_res = increase_test_background(False)  # many small increase vs one big one
    is_res = increase_test_sensitive(False)   # many small increase vs one big one
    s_res = small_test(False)                 # increase from small sensitive worse
    
    print(model)
    print(c_res, w_res, b_res, ib_res, is_res, s_res)
    print("")

Direct-Split
True False False True False False

Direct-Split(Tau)
True True False False False False

Direct-Split(None-Direct)
True False False True False False

Direct-Split(None-Split(Tau))
True True False True False False

Split-Split
False False False True False False

Web
True True True True False False

Fingers
True True False True False True

Fingers2
True True False True False True

Fingers3
False True True False True True

Fingers4
True True True True False True

Fingers5
False True True False False False

Fingers6
True True True False True True



In [16]:
model = "Fingers4"

pnp = PNP.read_text(f"Synthetic/{model}.pnp")
reachability_graph = pnp.create_reachability_graph()
all_labels = set(arc.transition.label for arc in chain.from_iterable(state.outgoing_arcs for state in reachability_graph.states))-{'tau', 'None'}

_max_B = {}
_pr_l = {}
_pr_lb = {}

c_res = change_test(True)                # change in sensitive vs highest
print("\n")
w_res = worst_test(True)                 # worst case vs all cases
print("\n")
b_res = background_test(True)            # probability of the background matering
print("\n")
ib_res = increase_test_background(True)  # many small increase vs one big one
print("\n")
is_res = increase_test_sensitive(True)   # many small increase vs one big one
print("\n")
s_res = small_test(True)                # increase from small sensitive worse

print(model)
print(c_res, w_res, b_res, ib_res, is_res, s_res)
print("")

P(s) = Max_B(P(s|b)):
A

P(s) < Max_B(P(s|b)):
F
A
G
C
B
E
D

P(s2) < Max_B(P(s2|b) < p(s1) = Max_B(P(s1|b)):
S1: A, S2: F
S1: A, S2: D


P(b1) < min_(B-b1)(P(b)):
D
P(s|b1) > max_(B-b1)(P(s|b)):
B1: D, S: B


P(b1) < P(b2):
B1: F, B2: A
B1: F, B2: G
B1: F, B2: C
B1: F, B2: B
B1: F, B2: E
B1: A, B2: B
B1: A, B2: E
B1: G, B2: A
B1: G, B2: C
B1: G, B2: B
B1: G, B2: E
B1: C, B2: A
B1: C, B2: B
B1: C, B2: E
B1: E, B2: B
B1: D, B2: F
B1: D, B2: A
B1: D, B2: G
B1: D, B2: C
B1: D, B2: B
B1: D, B2: E

P(b1) < P(b2) & P(S|b1) = P(S|b2) != 0:
B1: F, B2: G, S:A
B1: F, B2: G, S:C
B1: F, B2: G, S:B
B1: F, B2: G, S:D
B1: F, B2: E, S:A
B1: F, B2: E, S:G
B1: F, B2: E, S:C
B1: F, B2: E, S:B
B1: F, B2: E, S:D
B1: A, B2: B, S:F
B1: A, B2: B, S:G
B1: A, B2: B, S:E
B1: G, B2: E, S:A
B1: G, B2: E, S:C
B1: G, B2: E, S:B
B1: G, B2: E, S:D
B1: C, B2: A, S:F
B1: C, B2: A, S:G
B1: C, B2: A, S:E
B1: C, B2: B, S:F
B1: C, B2: B, S:G
B1: C, B2: B, S:E
B1: D, B2: A, S:F
B1: D, B2: A, S:G
B1: D, B2: A, S:E
B1: D, B2: 

In [17]:
label = "G"
print([{background, pr_lb(label, background)} for background in all_labels-set([label])])

[{0.5, 'F'}, {0.3499999999999999, 'A'}, {0.34999999999999987, 'C'}, {0.3499999999999999, 'B'}, {'E', 0.5}, {0.34999999999999987, 'D'}]


In [18]:
background = "A"
print([{label, pr_lb(label, background)} for label in all_labels-set([background])])

[{0.20000000000000004, 'F'}, {'G', 0.3499999999999999}, {0.20000000000000004, 'C'}, {0.0, 'B'}, {'E', 0.5}, {0.0, 'D'}]


In [19]:
print([{label, pr_l(label)} for label in all_labels])

[{0.2, 'F'}, {0.4, 'A'}, {'G', 0.3499999999999999}, {0.38, 'C'}, {0.6, 'B'}, {'E', 0.5}, {0.18, 'D'}]


In [65]:
model = "Fingers6"

pnp = PNP.read_text(f"Synthetic/{model}.pnp")
reachability_graph = pnp.create_reachability_graph()
all_labels = set(arc.transition.label for arc in chain.from_iterable(state.outgoing_arcs for state in reachability_graph.states))-{'tau', 'None'}

_max_B = {}
_pr_l = {}
_pr_lb = {}

c_res = change_test(True)                # change in sensitive vs highest
print("\n")
w_res = worst_test(True)                 # worst case vs all cases
print("\n")
b_res = background_test(True)            # probability of the background matering
print("\n")
ib_res = increase_test_background(True)  # many small increase vs one big one
print("\n")
is_res = increase_test_sensitive(True)   # many small increase vs one big one
print("\n")
s_res = small_test(True)                # increase from small sensitive worse

print(model)
print(c_res, w_res, b_res, ib_res, is_res, s_res)
print("")

P(s) = Max_B(P(s|b)):
A

P(s) < Max_B(P(s|b)):
B
G
C
E
F
D

P(s2) < Max_B(P(s2|b) < p(s1) = Max_B(P(s1|b)):
S1: A, S2: G
S1: A, S2: F
S1: A, S2: D


P(b1) < min_(B-b1)(P(b)):
G
P(s|b1) > max_(B-b1)(P(s|b)):
B1: G, S: F
B1: G, S: E


P(b1) < P(b2):
B1: A, B2: B
B1: G, B2: B
B1: G, B2: A
B1: G, B2: C
B1: G, B2: E
B1: G, B2: F
B1: G, B2: D
B1: C, B2: B
B1: C, B2: A
B1: E, B2: B
B1: E, B2: A
B1: E, B2: C
B1: F, B2: B
B1: F, B2: A
B1: F, B2: C
B1: F, B2: E
B1: F, B2: D
B1: D, B2: B
B1: D, B2: A
B1: D, B2: C
B1: D, B2: E

P(b1) < P(b2) & P(S|b1) = P(S|b2) != 0:
B1: A, B2: B, S:G
B1: A, B2: B, S:E
B1: A, B2: B, S:F
B1: G, B2: E, S:B
B1: G, B2: E, S:A
B1: G, B2: E, S:C
B1: G, B2: E, S:D
B1: G, B2: F, S:B
B1: G, B2: F, S:A
B1: G, B2: F, S:C
B1: G, B2: F, S:D
B1: C, B2: B, S:G
B1: C, B2: B, S:E
B1: C, B2: B, S:F
B1: C, B2: A, S:G
B1: C, B2: A, S:E
B1: C, B2: A, S:F
B1: F, B2: E, S:B
B1: F, B2: E, S:A
B1: F, B2: E, S:G
B1: F, B2: E, S:C
B1: F, B2: E, S:D
B1: D, B2: B, S:G
B1: D, B2: B, S:E
B1: D,

In [66]:
print(pr_lb("D", "B"))
print(pr_lb("F", "B"))

0.3
0.15544041450777202


In [52]:
for state in reachability_graph.states:
    print(state)
    print(state.outgoing_arcs)
    print("")

RS)P)n0:1
[A) RS)P)n0:1 -A,0.4> RS)P)n1:1, A) RS)P)n0:1 -B,0.6> RS)P)n2:1]

RS)P)n5:1
[]

RS)P)n1:1
[A) RS)P)n1:1 -C,0.2> RS)P)n3:1, A) RS)P)n1:1 -tau,0.8> RS)P)n3:1]

RS)P)n2:1
[A) RS)P)n2:1 -tau,0.2> RS)P)n3:1, A) RS)P)n2:1 -C,0.5> RS)P)n3:1, A) RS)P)n2:1 -D,0.3> RS)P)n3:1]

RS)P)n3:1
[A) RS)P)n3:1 -tau,0.4> RS)P)n5:1, A) RS)P)n3:1 -E,0.45> RS)P)n4:1, A) RS)P)n3:1 -F,0.15> RS)P)n4:1]

RS)P)n4:1
[A) RS)P)n4:1 -tau,0.52> RS)P)n3:1, A) RS)P)n4:1 -tau,0.28> RS)P)n5:1, A) RS)P)n4:1 -G,0.2> RS)P)n5:1]

